<a href="https://colab.research.google.com/github/YuktiMohan/Tensorflow-in-Practice/blob/master/Lesson%203%3A%20Natural%20Language%20Processing/3B%3A%20Word%20Embedding/Visualization_of_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL4FNZZ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL4FNZZ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL4FNZZ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train, test = imdb["train"], imdb["test"]

training_sentences = []
testing_sentences = []

training_labels = []
testing_labels = []

for s, l in train:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s, l in test:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

final_training_labels = np.array(training_labels)
final_testing_labels = np.array(testing_labels)

In [0]:
vocab_size = 10000
trunc_type = 'post'
embedding_dim = 16
oov_tok = "<007>"
max_len = 120

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_train = pad_sequences(training_sequences, maxlen = max_len, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_test = pad_sequences(testing_sequences, maxlen = max_len, truncating = trunc_type)

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded_train[1]))
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <007> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <007> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss= "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
epochs = 10
model.fit(padded_train, final_training_labels, epochs = epochs, validation_data=(padded_test, final_testing_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4868 - accuracy: 0.7494 - val_loss: 0.3816 - val_accuracy: 0.8284
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2433 - accuracy: 0.9054 - val_loss: 0.4138 - val_accuracy: 0.8182
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1017 - accuracy: 0.9736 - val_loss: 0.4989 - val_accuracy: 0.8090
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0265 - accuracy: 0.9960 - val_loss: 0.6110 - val_accuracy: 0.7993
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0061 - accuracy: 0.9996 - val_loss: 0.6862 - val_accuracy: 0.8045
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.7435 - val_accuracy: 0.8068
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.6972e-04 - accuracy: 1.0000 - val_loss: 0.7918 - val_accuracy: 0.8073
Ep

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
#code to make files for vectors and metadata.
#Vectors are the projections visualised on a Tensorflow projector

import io

out_v = io.open("vecs.tsv", 'w', encoding = 'utf-8')  #v -> vector
out_m = io.open("meta.tsv", 'w', encoding = 'utf-8')  #m -> metadata

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join(str(x) for x in embeddings) + '\n')

out_v.close()
out_m.close()

In [0]:
#code to download the above created files.

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')
